### ספריות

In [28]:
import os
import sys
import fiona
import pandas as pd
import geopandas as gpd
from openpyxl import load_workbook

In [29]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### העלת משתנים להרצת הקוד

In [30]:
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]

output_location=df_inputs_outputs['location'][2]

create_forecast_basic_software_location = os.path.dirname(software_folder_location)

sys.path.append(create_forecast_basic_software_location)

create_forecast_basic_software_df = pd.read_excel(r'{}\inputs_outputs.xlsx'.format(create_forecast_basic_software_location))

TAZ_V4_date = create_forecast_basic_software_df['location'][1]

### פונקציות גלובליות

In [31]:
from global_functions import up_load_shp, up_load_df, drop_geo, make_point, up_load_gdb, change_cols_names,add_geo_info_gdb,add_geo_info_shp

ממשיך את הקוד עכשיו שיכלול גם את המאפיינים הגיאוגרפים שעיסא צריך

In [32]:
col=['Taz_num','geometry']

In [33]:
taz_border=up_load_shp(r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_software_location, TAZ_V4_date))[col]

In [34]:
forecast_point = make_point(taz_border)

# Load data layers

urban = up_load_gdb(
r'{}\background_files\GIS_jtmt_forcast_v_3_2_Published.gdb'.format(software_folder_location), 'TAZ_V3_2_220123_urban')

# Geographical join between traffic zones and data layers

forecast_point_urban = forecast_point.sjoin(
urban)[['Taz_num', 'BaseProjections2040_csv_urban']]



In [35]:

# Merge tables into one table
taz_border = (taz_border
        .merge(forecast_point_urban, on='Taz_num', how='left'))

# Rename columns# Rename columns with flipped names

taz_border.rename(columns={'BaseProjections2040_csv_urban': 'urban'}, inplace=True)



In [36]:
#העלה של נתוני אנשים התפלגות גילים שנוצר לפני הקוד הזה
path=r'{}\Intermediates'.format(software_folder_location)
taz=up_load_df(path,'taz_before_add_geo')

In [37]:
taz['tazSector']=1 #ערבי

In [38]:
taz.loc[taz['main_secto']=='U_Orthodox','tazSector']=2

In [39]:
taz.loc[taz['main_secto']=='Jewish','tazSector']=3

In [40]:
taz.loc[taz['main_secto']=='Palestinian','tazSector']=4

לייצר פומה

עיבוד של הטבלה כדי שיהיה אפשר לחבר לכל אזור תנועה את הערך הנכון

In [41]:
poly_pumas=up_load_shp(r'{}\background_files\poly_pumas.shp'.format(software_folder_location))

In [42]:
col_old=['poly_puma',  'F3', 'F2', 'F1', 'geometry']

In [43]:
col_new=['poly_puma',  '3', '2', '1', 'geometry']

In [44]:
poly_pumas=drop_geo(change_cols_names(poly_pumas,col_old,col_new))

In [45]:
pumas_by_poly_sector=poly_pumas.melt(id_vars='poly_puma',var_name='tazSector',value_name='PUMA')

In [46]:
pumas_by_poly_sector['tazSector']=pumas_by_poly_sector['tazSector'].astype(int)

In [47]:
taz=add_geo_info_shp(taz,taz_border,software_folder_location,'poly_pumas','poly_puma')

In [48]:
taz=taz.merge(pumas_by_poly_sector,on=['poly_puma','tazSector'],how='left')

In [49]:
taz.loc[taz['PUMA']==0,'PUMA']=999

In [50]:
taz.loc[taz['pop']==0,'PUMA']=999

In [51]:
taz.loc[taz['main_secto']=='Palestinian','PUMA']=999

In [52]:
#col=['geometry','main_secto','Muni_Heb','poly_puma','PUMA','pop']
#taz_border[['Taz_num', 'geometry']].merge(drop_geo(taz),on='Taz_num',how='left')[col].to_file(r'puma_for_amirs_check.shp')

In [53]:
taz.loc[taz['jeru_metro']==0,'PUMA']=999

In [54]:
# Add constant columns
taz_border['REGION'] = 1
taz_border['slop'] = 0

אזורי חינוך

In [55]:
taz=add_geo_info_shp(taz,taz_border,software_folder_location,'schDistrict','ID').rename(columns={'ID':'SCHOOLDISTRICT'})

In [56]:
taz.loc[taz['SCHOOLDISTRICT']==0,'SCHOOLDISTRICT']=999

להעלות את המידע שנעשה עד עכשיו כדי לייצא פורמט שעיסא רוצה

In [57]:
taz=taz.merge(drop_geo(taz_border),on='Taz_num',how='left')

In [58]:
taz['yosh']=0

taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

taz['jerusalem_city']=0

taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

taz=taz.reset_index()

פלט של ההיברדי

In [59]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [60]:

col_needed=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slop',
'urban']

col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed',
'slop',
'urban']

In [61]:
#### תאריך
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [62]:
change_cols_names(taz, col_needed, col_new_name).to_excel(r'{}\BaseProjections2020_{}.xlsx'.format(output_location,file_date),index=False)

In [63]:
taz['DISTRICT']=999

In [64]:
taz.loc[taz['jew']==0,'DISTRICT']=1
taz.loc[(taz['main_secto']=='U_Orthodox')&(taz['in_jerusal']==1),'DISTRICT']=2
taz.loc[(taz['main_secto']=='Jewish')&(taz['in_jerusal']==1),'DISTRICT']=3
taz.loc[(taz['main_secto']=='Jewish')&(taz['in_jerusal']==0)&(taz['jeru_metro']==1),'DISTRICT']=5
taz.loc[(taz['main_secto']=='U_Orthodox')&(taz['in_jerusal']==0)&(taz['jeru_metro']==1),'DISTRICT']=6

In [65]:
taz.loc[taz['pop']==0,'DISTRICT']=999

In [66]:

col_needed=['Taz_num','Agg_taz_nu','PUMA','DISTRICT','REGION','SCHOOLDISTRICT']

col_new_name=['TAZ','AGG_TAZ','PUMA','DISTRICT','REGION','SCHOOLDISTRICT']

In [67]:
change_cols_names(taz, col_needed, col_new_name).to_excel(r'{}\puma2020_{}.xlsx'.format(output_location,file_date),index=False)

פלט של הנוסף שצריך בשביל הABM

In [68]:
col_with_fix_num=['area',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'county',
 'majunivenr',
 'parktot',
 'superZone',
 'Taz1']

In [69]:
taz[col_with_fix_num]=1

In [70]:
taz['UNIVENRORTHFEMALE']=taz['Univ_UO']/2
taz['UNIVENRORTHMALE']=taz['Univ_UO']/2


In [71]:
taz['TOA1']=taz['SEA1']+taz['UOA1']+taz['ARA1']
taz['TOA2']=taz['SEA2']+taz['UOA2']+taz['ARA2']
taz['TOA3']=taz['SEA3']+taz['UOA3']+taz['ARA3']

In [72]:
forecast_point = make_point(taz_border)

# Load data layers
hibiz = up_load_shp(r'{}\background_files\highBusinessFlag.shp'.format(software_folder_location))[['HighBiz', 'geometry']]


In [73]:

forecast_point_hibiz = forecast_point.sjoin(hibiz)[['Taz_num', 'HighBiz']]

taz = taz.merge(forecast_point_hibiz, on='Taz_num', how='left')

taz.rename(columns={'HighBiz': 'highBusinessFlag'}, inplace=True)


In [74]:
col_parking=['FreeBuffer',
'PaidBuffer',
'Rest_EmpBuffer',
'searchtime',
'walktime',
'cost']

In [75]:
for c in col_parking:
    taz=add_geo_info_gdb(taz,taz_border,software_folder_location,'parking_abm',c,c)

In [76]:
col_emp_EI=['EIProp']

In [77]:
for c in col_emp_EI:
    taz=add_geo_info_gdb(taz,taz_border,software_folder_location,'emp_EI',c,c)

In [78]:
taz['perScaled']=1-taz['EIProp']

In [79]:
taz['ieold']=0
taz.loc[taz['jeru_metro']==1,'ieold']=taz['commuting']
taz['IEProp']=taz['ieold']

הייצוא עצמו לטובת הSED FILE

In [80]:
col_needed=['Taz_num',
 'Taz_num',
 'hh',
 'PUMA',
 'DISTRICT',
 'county',
 'area',
 'parktot',
 'majunivenr',
 'tazSector',
 'Indus',
 'Com_hotel',
 'Business',
 'Public',
 'emp_Education',
 'agri',
 'total_emp',
 'UOA1',
 'UOA2',
 'UOA3',
 'SEA1',
 'SEA2',
 'SEA3',
 'ARA1',
 'ARA2',
 'ARA3',
 'TOA1',
 'TOA2',
 'TOA3',
 'Univ_AR',
 'Univ_SE',
 'UNIVENRORTHMALE',
 'UNIVENRORTHFEMALE',
 'ieold',
 'superZone',
 'IEProp',
 'Taz1',
 'perScaled',
 'EIProp',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'PaidBuffer',
 'Rest_EmpBuffer',
 'FreeBuffer',
 'SCHOOLDISTRICT',
 'SCHOOLDISTRICT',
 'highBusinessFlag',
 'searchtime',
 'walktime',
 'cost']

In [81]:
col_new_name=['maz',
 'taz',
 'hh_total',
 'puma',
 'district',
 'county',
 'area',
 'parktot',
 'majunivenr',
 'tazSector',
 'Indus',
 'Com_hotel',
 'Off_Bsness',
 'Public',
 'Education',
 'Agri',
 'totemp',
 'UOA1',
 'UOA2',
 'UOA3',
 'SEA1',
 'SEA2',
 'SEA3',
 'ARA1',
 'ARA2',
 'ARA3',
 'TOA1',
 'TOA2',
 'TOA3',
 'UNIVENRARAB',
 'UNIVENRSEC',
 'UNIVENRORTHMALE',
 'UNIVENRORTHFEMALE',
 'ieold',
 'superZone',
 'IEProp',
 'Taz1',
 'perScaled',
 'EIProp',
 'CITYCODE1',
 'CITYCODE2',
 'CITYCODE3',
 'CITYCODE4',
 'codeseq',
 'codeseqCons',
 'PaidBuffer',
 'Rest_EmpBuffer',
 'FreeBuffer',
 'schDistrict',
 'schDistrictAgg',
 'highBusinessFlag',
 'searchtime',
 'walktime',
 'cost']

In [82]:
change_cols_names(taz, col_needed, col_new_name).to_excel(r'{}\SED_2020_{}.xlsx'.format(output_location,file_date),index=False)

In [110]:
taz.to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(output_location,file_date),index=False)

In [83]:
#load excel file
workbook = load_workbook(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))

#open workbook
sheet = workbook.active

#modify the desired cell
sheet["B4"] = forecast_version_folder_location
sheet["B5"] = False
sheet["B6"] = r'{}\background_files\TAZ_V4_{}_with_geo_info.shp'.format(create_forecast_basic_software_location, TAZ_V4_date)

#save the file
workbook.save(filename=r'{}\inputs_outputs.xlsx'.format(software_folder_location))